In [1]:

import pandas as pd
from scipy import sparse
import sys
import pathlib
import numpy as np
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.TopicModeling.tm_utils.prompter import Prompter
from src.TopicModeling.tm_utils.cpv_codes import CPV_CODES

/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_data = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Text_Object/data/train_data/processed_objectives_with_lemmas_emb_label.parquet")
TMfolder = pathlib.Path("/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/sample_data/cpv_models_iter2/0_45/7_topics/model_data/TMmodel")
cpv = TMfolder.parent.parent.parent.stem.split("/")[-1]
cpv_file = path_data / f"{cpv}.parquet"
df_all = pd.read_parquet(path_data)
cpv_code = cpv.split("_")[-1]
#cpv_label = CPV_CODES[cpv_code]
#cpv_label
df_all

,id,df_origin,place_id,link,procurement_id,expediente,objective,orig_objective,title,lang_objective,similarity,origin,raw_text,lemmas,embeddings,date,cpv,raw_textc,two_cpv,predicted_label
0,0,df_gencat,https://contrataciondelestado.es/sindicacion/P...,<NA>,ntp00114200,None,El objeto del presente Pliego es la definición...,None,Suministro de Detector de Equipo de RX Portáti...,es,0.641474,out,El objeto del presente Pliego es la definición...,presente pliego prescripciones_técnicas sumini...,-1.0870938e-05 -0.07971101 -0.010653946 0.0071...,None,0 ...,None,[33],1
1,1,df_gencat,https://contrataciondelestado.es/sindicacion/l...,<NA>,ntp00729859,None,"El objeto del contrato, correspondiente a la p...",None,Prestación de Servicios de Explotación de los ...,es,0.247979,ins,"El objeto del contrato, correspondiente a la p...",presente mutual_midat cyclops_mutua colaborado...,0.035840757 -0.0459068 -0.006238037 -0.0255564...,None,0 ...,None,[50],0
2,2,df_gencat,https://contrataciondelestado.es/sindicacion/l...,<NA>,ntp01335443,None,El objeto de estos pliegos es la definición de...,None,Servicio de gestión de las oficinas municipale...,es,0.657591,ins,El objeto de estos pliegos es la definición de...,específico servicio integral oficinas municipa...,-0.017299483 -0.01512944 -0.0015853408 -0.0080...,None,0 ...,None,[63],1
3,3,df_gencat,https://contrataciondelestado.es/sindicacion/P...,<NA>,ntp00196761,None,El presente Pliego de Prescripciones tiene por...,None,Servicios por lotes para la conservación de vi...,es,0.567718,out,El presente Pliego de Prescripciones tiene por...,presente pliego prescripciones fijar servicio ...,0.015432008 0.016543098 0.015060815 -0.0115206...,None,0 ...,None,"[90, 60]",1
4,4,df_gencat,https://contrataciondelestado.es/sindicacion/l...,<NA>,ntp01108524,None,Contratación del servicio de profesional de re...,None,Representación judicial y defensa jurídica en ...,es,0.870487,ins,Contratación del servicio de profesional de re...,servicio representación_procesal jurídico mutu...,0.0042968676 0.009542765 0.008155834 -0.003156...,None,0 ...,None,[79],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437830,437830,zaragoza,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,None,0009891-23,contratar la asistencia técnica en materia de ...,Servicios de consultoría en materia de segurid...,Servicio de asistencia técnica en materia de s...,es,0.813420,ins,contratar la asistencia técnica en materia de ...,contratar materia mantenimiento servicio explo...,-0.008637608 -0.018740915 -0.018331034 -0.0221...,2024-05-02,0 ...,"D Taragoza Ane de infraestructuras, vivienda Y...",[71],1
437831,437831,zaragoza,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,None,0027384-23,Suministro de Placas de cierre para las distin...,Suministro de placas de cierre para las distin...,Suministro de placas de cierre para el Cemente...,es,0.825017,ins,Suministro de Placas de cierre para las distin...,suministro placa entrenamiento cementerio muni...,-0.06869736 0.013652654 0.028806146 0.04934917...,2023-07-21,0 ...,que Zara goza servicio DE tNFoRMActóru v trénc...,[44],1
437832,437832,zaragoza,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,None,0093023-22,Es objeto de la presente licitación la convoca...,Explotación y mantenimiento integral del servi...,Explotación de las instalaciones de recogida n...,es,0.886649,ins,Es objeto de la presente licitación la convoca...,presente abierto explotación mantenimiento int...,-0.035525475 -0.031006623 -0.008097607 0.00416...,2023-04-25,0 ...,G es+ Taragoza Ayuno y Alto Servicio EL ESPACI...,[90],1
437833,437833,zaragoza,https://contrataciondelestado.es/sindicacion/l...,https://contrataciondelestado.es/wps/poc?uri=d...,None,0062395-22,el objeto de este contrato es el suministro e ...,Suministro e instalación de equipamiento

In [3]:
corpusFile = TMfolder.parent.parent.joinpath('train_data/corpus.txt')
with corpusFile.open("r", encoding="utf-8") as f:
    lines = f.readlines()  
    f.seek(0)
    try:
        documents_ids = [line.rsplit(" 0 ")[0].strip() for line in lines]
        documents_texts = [line.rsplit(" 0 ")[1].strip().split() for line in lines]
    except:
        documents_ids = [line.rsplit("\t0\t")[0].strip() for line in lines]
        documents_texts = [line.rsplit("\t0\t")[1].strip().split() for line in lines]
df_corpus_train = pd.DataFrame({'id': documents_ids, 'text': documents_texts})
df_corpus_train = pd.merge(df_corpus_train, df_all[['place_id', 'procurement_id','raw_text']], left_on='id', right_on = "place_id", how='inner')
df_corpus_train

,id,text,place_id,procurement_id,raw_text
0,https://contrataciondelestado.es/sindicacion/l...,"[desmontaje, saneamiento, estación_depuradora_...",https://contrataciondelestado.es/sindicacion/l...,ntp00688421,El objeto de este proyecto es el desmontaje y ...
1,https://contrataciondelestado.es/sindicacion/P...,"[reforma, oficina, mantenimiento, energía, san...",https://contrataciondelestado.es/sindicacion/P...,ntp00340338,El objeto del presente pliego de prescripcione...
2,https://contrataciondelestado.es/sindicacion/P...,"[reforma, oficina, mantenimiento, energía, san...",https://contrataciondelestado.es/sindicacion/P...,ntp00356080,El objeto del presente pliego de prescripcione...
3,https://contrataciondelestado.es/sindicacion/d...,"[reparación, cubierta, edificio, instalar, aérea]",https://contrataciondelestado.es/sindicacion/d...,ntp11378769,El objeto del presente Pliego de Prescripcione...
4,https://contrataciondelestado.es/sindicacion/l...,"[hormigón, reparación, interior, perimetral, c...",https://contrataciondelestado.es/sindicacion/l...,ntp01189449,La actuación a realizar consiste en la ejecuci...
...,...,...,...,...,...
36706,https://contrataciondelestado.es/sindicacion/l...,"[n, térmico, compuesto, térmico, servicio, cal...",https://contrataciondelestado.es/sindicacion/l...,None,ANTECEDENTES Y OBJETO Actualmente el PDM Palac...
36707,https://contrataciondelestado.es/sindicacion/l...,"[reforma, conservación, demolición, edificio, ...",https://contrataciondelestado.es/sindicacion/l...,None,la definición de las condiciones que regirán e...
36708,https://contrataciondelestado.es/sindicacion/l...,"[restauración, procedente, salón, antiguo]",https://contrataciondelestado.es/sindicacion/l...,None,El objeto del proyecto es la restauración de u...
36709,https://contrataciondelestado.es/sindicacion/l...,"[pavimento, reparación, escollera, reparación,...",https://contrataciondelestado.es/sindicacion/l...,None,Renovación de pavimentos de andadores Renovaci...


In [4]:
df_corpus_train.id.nunique()

23402

In [5]:
def get_most_representative_per_tpc(mat, topn=3):
    # Find the most representative document for each topic based on a matrix mat
    top_docs_per_topic = []
    
    for doc_distr in mat.T:
        sorted_docs_indices = np.argsort(doc_distr)[::-1]
        top = sorted_docs_indices[:topn].tolist()
        top_docs_per_topic.append(top)
    return top_docs_per_topic

In [6]:
s3 = sparse.load_npz(TMfolder /  "s3.npz").toarray()   
top_docs_per_topic = get_most_representative_per_tpc(s3, topn=3)

In [7]:
documents_ids[20148]

'https://contrataciondelestado.es/sindicacion/licitacionesPerfilContratante/12802729'

In [8]:
with open(TMfolder  / "tpc_descriptions.txt", 'r') as file: lines = file.readlines()
topic_keys = [line.strip() for line in lines]
topic_keys

['tramo, pavimentación, carretera, pavimento, urbanización, asfaltado, firme, peatonal, núcleo, viario, vehículo, peatón, reparación, tráfico, rodadura',
 'climatización, planta, reforma, edificio, oficina, enegía, equipo, aseo, ascensor, asistencial, residencia, rehabilitación, aula, interior, térmico',
 'mantenimiento, conservación, alquiler, servicio, equipo, preventivo_correctivo, desmontaje, evento, correctivo, carpa, reparación, eléctrico, deportivo, vigilancia, climatización',
 'cubierta, fachada, reparación, edificio, conservación, planta, patio, restauración, pintura, cerramiento, rehabilitación, ventana, sustitución, pavimento, interior',
 'deportivo, fútbol, pavimento, pista, infantil, polideportivo, césped_artificial, juego, parcela, juego_infantil, área_juego, vestuario, cubierta, pista_pádel, artificial',
 'abastecimiento, alumbrado, enegía, eléctrico, potable, saneamiento, tramo, depósito, energía, residual, canalización, tensión, conexión, luminaria, conducción',
 'vehí

In [18]:
prompter =  Prompter(model_type="gpt-4o-mini-2024-07-18", config_path="/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/config/prompter_config.yaml")
INSTRUCTIONS_PATH = "/export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/src/TopicModeling/tm_utils/prompts/labelling_with_cpv.txt"
with open(INSTRUCTIONS_PATH, 'r') as file: TEMPLATE = file.read()

for tpc_id, tpc in enumerate(topic_keys):
    topc_docs = top_docs_per_topic[tpc_id]
    docs = "\n- " + "\n- ".join([f"{doc} - {df_corpus_train[df_corpus_train.id == documents_ids[doc]].raw_text.values[0]}" for doc in topc_docs])
    template = TEMPLATE.format(keywords=tpc, docs = docs, cpv="Trabajos de construcción")
    label, _ = prompter.prompt(question=template, system_prompt_template_path=None)

    print(f"\033[92mLABEL: {label}\033[0m")

Loaded config file /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/config/prompter_config.yaml and section logger.
Logs will be saved in data/logs


src.TopicModeling.tm_utils.prompter - INFO - Using OpenAI API with model: gpt-4o-mini-2024-07-18


Loaded config file /export/usuarios_ml4ds/lbartolome/NextProcurement/NP-Search-Tool/config/prompter_config.yaml and section llm.
Cache key: c4ca9a7612e768722c972d124ef8d96c
Cache miss: computing results...
LABEL: Infraestructura vial
Cache key: c86b19d6bc4ba6f98524495c13cfe8b9
Cache miss: computing results...
LABEL: Rehabilitación de edificios
Cache key: c6145b7d00bc7cc24e67f18b0f2305db
Cache miss: computing results...
LABEL: Servicios de mantenimiento y conservación
Cache key: 26d6aee60aef76593ddd9cfb54bf9d30
Cache miss: computing results...
LABEL: Rehabilitación y conservación de edificios
Cache key: d5b902eb822cb88afbca6bc8d5bff9a5
Cache miss: computing results...
LABEL: Infraestructura deportiva
Cache key: bf051f4f1f0c994f01b9844878704b00
Cache miss: computing results...
LABEL: Infraestructura de servicios públicos
Cache key: d868a2fc43ac163a4373295ecb7c90b9
Cache miss: computing results...
LABEL: Infraestructura de recarga para vehículos eléctricos
